In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import re
from fuzzywuzzy import fuzz

def get_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_damodaran/totalbeta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def process_df(x):
    columns = x.iloc[0,:]
    columns.name = ""
    x.columns = columns
    x = x.iloc[1:,:]
    x.reset_index(drop=True, names=columns, inplace=True)
    x.rename(index={"6":""}).head(7)
    return x

/Users/leonplaza/miniconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [84]:
def industry_parser():
    gics = pd.read_csv("Database/gics-map-2018.csv").iloc[:,[1,3,5,7]]
    industries = list(get_betas().index)

    match = []
    for i in industries:
        ls = []
        for k in list(gics.SubIndustry):
            ls.append([fuzz.ratio(i, k),i,k])
        x = pd.DataFrame(ls).sort_values(0, ascending=False)
        match.append(list(x.iloc[0,:]))

    match = pd.DataFrame(match, columns=["fuzz subind", "damodaran", "subindustry"])
    match.set_index("damodaran",inplace=True)
    match["tup subind"] = [(match.iloc[i,0],match.iloc[i,1]) for i in range(len(match))]
    df_subind = match["tup subind"]

    match = []
    for i in industries:
        ls = []
        for k in list(gics.Industry):
            ls.append([fuzz.ratio(i, k),i,k])
        x = pd.DataFrame(ls).sort_values(0, ascending=False)
        match.append(list(x.iloc[0,:]))

    match = pd.DataFrame(match, columns=["fuzz ind", "damodaran", "industry"])
    match.set_index("damodaran",inplace=True)
    match["tup industry"] = [(match.iloc[i,0],match.iloc[i,1]) for i in range(len(match))]
    df_ind = match["tup industry"]

    match = []
    for i in industries:
        ls = []
        for k in list(gics.IndustryGroup):
            ls.append([fuzz.ratio(i, k),i,k])
        x = pd.DataFrame(ls).sort_values(0, ascending=False)
        match.append(list(x.iloc[0,:]))

    match = pd.DataFrame(match, columns=["fuzz indgroup", "damodaran", "industry group"])
    match.set_index("damodaran", inplace=True)
    match["tup indgroup"] = [(match.iloc[i,0],match.iloc[i,1]) for i in range(len(match))]
    df_indgroup = match["tup indgroup"]

    x = pd.concat([df_subind, df_ind, df_indgroup],axis=1)
    ls = []

    for i in industries:
        a = max([x.loc[i,"tup subind"][0],x.loc[i,"tup industry"][0],x.loc[i,"tup indgroup"][0]])
        if x.loc[i,"tup subind"][0] == a:      
            ls.append([i,x.loc[i,"tup subind"][1]])
        elif x.loc[i,"tup industry"][0] == a:      
            ls.append([i,x.loc[i,"tup industry"][1]])
        elif x.loc[i,"tup indgroup"][0] == a:      
            ls.append([i,x.loc[i,"tup indgroup"][1]])
        else:
            pass

    x = pd.DataFrame(ls).rename({0:"Damodaran",1:"GICS"},axis=1)#.set_index(1).loc[:,0]
    #industry = yf.Ticker(ticket).info.get("industry")
    return x#[x["GICS"] == "Consumer Electronics"]

In [86]:
%%time
industry_parser().iloc[10:60,:]

CPU times: user 1.15 s, sys: 6.98 ms, total: 1.16 s
Wall time: 1.15 s


,Damodaran,GICS
10,Broadcasting,Broadcasting
11,Brokerage & Investment Banking,Investment Banking & Brokerage
12,Building Materials,Materials
13,Business & Consumer Services,Consumer Services
14,Cable TV,Cable & Satellite
15,Chemical (Basic),Chemicals
16,Chemical (Diversified),Chemicals
17,Chemical (Specialty),Chemicals
18,Coal & Related Energy,Cable & Satellite
19,Computer Services,Consumer Services


In [83]:
x = pd.read_csv("Database/gics-map-2018.csv")
x[x["Sector"] == "Software"]

yf.Ticker("AAPL").info.get("industry")

'Consumer Electronics'

In [13]:
def get_industry(ticket):
    ls = []
    ind = yf.Ticker(ticket).info.get("industry")
    for i in industries:
        ls.append([fuzz.ratio(ind, i),i])
    return pd.DataFrame(ls).sort_values(0, ascending=False).iloc[0,1]

In [14]:
%%time
ls = []
for i in industries:
    ls.append([fuzz.ratio(yf.Ticker("MSFT").info.get("industry"), i),i])
pd.DataFrame(ls).sort_values(0, ascending=False).iloc[0,1]


CPU times: user 603 ms, sys: 96.5 ms, total: 699 ms
Wall time: 6.33 s


'Software (Internet)'